In [1]:
from future.backports.datetime import datetime
from misc.misc import run_in_thread
%load_ext autoreload
%autoreload 2
import matplotlib
matplotlib.use('TKAgg')
from pianoq.lab.mplc.mask_aligner import MPLCAligner
from pianoq.lab.pco_camera import PCOCamera
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.mplc_device import MPLCDevice
import os 
import datetime
import time 
dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import glob 
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots
import numpy as np 
modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])

mplc = MPLCDevice()
mplc.restore_location()
print('Got MPLC')
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_10_14_00_34_old.masks"
masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_25_13_19_15_extra_1cm.masks"
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_17_first_diffuser\2024_09_17_12_19_58_diffuser_d1d.masks"

msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
phases = np.random.uniform(0, 2*np.pi, size=50)
masks = add_phase_input_spots(masks, phases)
mplc.load_masks(masks, linear_tilts=True)
mplc.restore_location()

Got MPLC


In [3]:
backlash = 0
wait_after_move = 0.3
from pianoq.lab.mplc.consts import thorlabs_x_serial, thorlabs_y_serial
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.zaber_motor import ZaberMotors
zaber_ms = ZaberMotors(backlash=backlash, wait_after_move=wait_after_move)
mxs = zaber_ms.motors[1]
mys = zaber_ms.motors[0]
print('Got Zaber motors')

# mxi = ThorlabsKcubeDC(thorlabs_x_serial, backlash=backlash, wait_after_move=wait_after_move)
# myi = ThorlabsKcubeStepper(thorlabs_y_serial, backlash=backlash, wait_after_move=wait_after_move)
# print('Got Thorlabs motors')

Got Zaber motors


In [4]:
# Timetagger
from pianoq.lab.time_tagger import QPTimeTagger
from pianoq.lab.mplc.consts import TIMETAGGER_DELAYS, TIMETAGGER_COIN_WINDOW
tt = QPTimeTagger(integration_time=1, remote=True,
                  single_channel_delays=TIMETAGGER_DELAYS, coin_window=TIMETAGGER_COIN_WINDOW)

In [44]:
mxs.move_absolute(12.99)
mys.move_absolute(6.79)
print(mxs.get_position())
print(mys.get_position())

12.990004500000001
6.7899915


In [49]:
from pianoq.lab.photon_scan import PhotonScanner
from pianoq.misc.misc import run_in_thread

integration_time = 1
start_x = 12.65
end_x = 13.35
start_y = 4.8
end_y = 7.0

mxs.wait_after_move = 0.1
mys.wait_after_move = 0.1

resolution = 1 
x_pixels = 14 // resolution
y_pixels = 44 // resolution
pixel_size_x = 0.05 * resolution
pixel_size_y = 0.05 * resolution

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
singles_signal_path = fr'{dir_path}\{timestamp}_signal_3.5_spots_tilt_20.scan'

scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=2e-9, saveto_path=singles_signal_path)   
_ = scanner.scan(x_motor=mxs, y_motor=mys, ph=tt)

Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 249. Singles2: 648. Coincidence: 0.00.
dur: 1. pix: 0, 1. Singles1: 320. Singles2: 639. Coincidence: 0.00.
dur: 2. pix: 0, 2. Singles1: 319. Singles2: 663. Coincidence: 0.00.
dur: 4. pix: 0, 3. Singles1: 342. Singles2: 655. Coincidence: 0.00.
dur: 5. pix: 0, 4. Singles1: 430. Singles2: 672. Coincidence: 0.00.
dur: 7. pix: 0, 5. Singles1: 343. Singles2: 652. Coincidence: 0.00.
dur: 8. pix: 0, 6. Singles1: 403. Singles2: 711. Coincidence: 0.00.
dur: 9. pix: 0, 7. Singles1: 370. Singles2: 698. Coincidence: 0.00.
dur: 11. pix: 0, 8. Singles1: 412. Singles2: 719. Coincidence: 0.00.
dur: 12. pix: 0, 9. Singles1: 421. Singles2: 681. Coincidence: 0.00.
dur: 13. pix: 0, 10. Singles1: 573. Singles2: 726. Coincidence: 0.00.
dur: 15. pix: 0, 11. Singles1: 619. Singles2: 688. Coincidence: 0.00.
dur: 16. pix: 0, 12. Singles1: 523. Singles2: 635. Coincidence: 0.00.
dur: 18. pix: 0, 13. Singles1: 428. Singles2: 627. Coincidence: